In [1]:
import cv2
import numpy as np
import concurrent.futures
import albumentations as A

def identify_seam_photos(image, dynamic_threshold_factor=0.5):
    # Convert the image to grayscale
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Compute the dynamic threshold based on image characteristics
    dynamic_threshold = np.mean(grayscale_image) * dynamic_threshold_factor
    
    # Apply thresholding to identify seam photos
    _, binary_image = cv2.threshold(grayscale_image, dynamic_threshold, 255, cv2.THRESH_BINARY)
    
    # Create a 3-channel binary image for blending
    binary_image_colored = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2BGR)
    
    # Create a mask where the binary image is black
    mask = binary_image == 0
    
    # Blend the original image with the binary image
    blended_image = np.copy(image)
    blended_image[mask] = binary_image_colored[mask]
    
    return blended_image

def resize_frame(frame, frame_width=720, frame_height=800):
    # Resize the frame to the desired size
    resized_frame = cv2.resize(frame, (frame_width, frame_height))
    return resized_frame

def augment_frame(frame):
    # Define smoother augmentation pipeline
    augmentations = A.Compose([
       
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),  # Subtle brightness/contrast changes
        A.GaussNoise(var_limit=(10.0, 20.0), p=0.5),  # Reduced noise variance
        A.MotionBlur(blur_limit=3, p=0.5),  # Smaller kernel for motion blur
        A.GridDistortion(num_steps=5, distort_limit=0.1, p=0.5),  # Subtle grid distortions
        A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=0.5)  # Reduced optical distortion
    ])
    augmented = augmentations(image=frame)['image']
    return augmented

def process_frame(frame, dynamic_threshold_factor=0.5):
    if frame is None:
        return None, None

    # Resize the frame
    resized_frame = resize_frame(frame)

    # Apply augmentations
    augmented_frame = augment_frame(resized_frame)

    # Identify seam photos using dynamic thresholding on the augmented frame
    seam_photos = identify_seam_photos(augmented_frame, dynamic_threshold_factor)
    
    return seam_photos, resized_frame

def process_video(video_path, max_workers=4, frame_width=720, frame_height=800):
    # Initialize the video capture
    cap = cv2.VideoCapture(video_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Process the frame using ThreadPoolExecutor
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            future = executor.submit(process_frame, frame)
            result, resized_frame = future.result()

        # Display the processed frame
        if result is not None:
            cv2.imshow('Seam Photos', result)
        else:
            print("No result to display for this frame")

        # Display the resized original frame
        cv2.imshow('Original Frame', resized_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Path to the video file
video_path = "0625.mp4"

# Process and display video frames
process_video(video_path)
